In [1]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json

# get_ticker 함수: 주어진 회사 이름을 입력받아 DuckDuckGo에서 해당 회사의 주식 심볼을 검색하고 반환.
def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")

# get_income_statement 함수: 주식 심볼을 입력받아, 해당 회사의 손익계산서를 yfinance에서 가져와 JSON 형식으로 반환.
def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())

# get_balance_sheet 함수: 주식 심볼을 입력받아 해당 회사의 대차대조표를 가져와 JSON 형식으로 반환.
def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())

# get_daily_stock_performance 함수: 주식 심볼을 입력받아 해당 회사의 3개월 간의 주식 성과를 가져와 JSON 형식으로 반환.
def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())


functions_map = {
    "get_ticker": get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_daily_stock_performance": get_daily_stock_performance,
}

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

In [2]:
# from dotenv import load_dotenv
# import os

# load_dotenv()
# api_key = os.getenv('OPENAI_API_KEY')

# from openai import OpenAI
# client = OpenAI()

# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#     model="gpt-4-1106-preview",
#     tools=functions,
# )
# assistant

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

from openai import OpenAI
client = OpenAI()


assistant_id = "asst_mcY4xDFQCioGKqiVFKLX6hrW"

In [4]:
# thread 생성: 사용자가 Salesforce 주식에 대해 물어본 대화를 시작하기 위해 새로운 쓰레드를 생성.
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know if the Salesforce stock is a good buy",
        }
    ]
)

In [5]:
# run 생성: assistant_id와 thread_id를 사용하여 대화 실행. 이로 인해 function_call이 실행될 수 있음.
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_Y0XA7nsD21XNu70OH2Z0woLB', assistant_id='asst_mcY4xDFQCioGKqiVFKLX6hrW', cancelled_at=None, completed_at=None, created_at=1726371137, expires_at=1726371737, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_7peUVHUyBE7iiF2VhsyYp6IA', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=Functio

In [6]:
# get_run 함수: 주어진 run_id와 thread_id에 대한 실행 정보를 가져오는 함수.
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )

# send_message 함수: 사용자로부터 새로운 메시지를 주어진 thread_id로 전송하는 함수.
def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )

# get_messages 함수: 주어진 thread_id의 모든 메시지를 가져와 출력.
def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")

# get_tool_outputs 함수: run_id와 thread_id에 있는 실행(run)에서 호출된 도구의 출력을 가져와 출력.
def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread.id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs

# submit_tool_outputs 함수: 가져온 도구의 출력 결과를 OpenAI API에 제출하여 실행 결과를 업데이트.
def submit_tool_outputs(run_id, thread_id):
    outpus = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id, thread_id=thread_id, tool_outputs=outpus
    )

In [15]:
submit_tool_outputs(run.id, thread.id)


Calling function: get_income_statement with arg {"ticker": "CRM"}
Calling function: get_balance_sheet with arg {"ticker": "CRM"}
Calling function: get_daily_stock_performance with arg {"ticker": "CRM"}


Run(id='run_Y0XA7nsD21XNu70OH2Z0woLB', assistant_id='asst_mcY4xDFQCioGKqiVFKLX6hrW', cancelled_at=None, completed_at=None, created_at=1726371137, expires_at=1726371737, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1726371217, status='queued', thread_id='thread_7peUVHUyBE7iiF2VhsyYp6IA', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=F

In [32]:
get_run(run.id, thread.id).status


'completed'

In [33]:
get_messages(thread.id)


user: I want to know if the Salesforce stock is a good buy
assistant: To evaluate whether Salesforce (CRM) stock is a good buy, we need to consider its financial health and stock performance. Here is a summary of Salesforce's recent financial statements and stock performance based on the data retrieved:

Income Statement Highlights (Most Recent Quarters):
- Total revenue has been increasing, with the most recent quarter showing \$34.857 billion.
- Gross profit has also seen an upward trend, with the last quarter reporting \$26.316 billion.
- Operating income and net income are positive, indicating profitability. The last quarter's net income was \$4.136 billion.

Balance Sheet Highlights (Most Recent Quarter):
- Total assets stand at \$99.823 billion, suggesting a substantial asset base.
- Total liabilities are significant at \$40.177 billion, but the company maintains a healthy equity position of \$59.646 billion.
- Goodwill and intangible assets make up a large portion of the assets 